# Exploring probabilistic modelling - Tensorflow probability and Iris dataset

* co-created by Stepohen Haddad and ChatGPT


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp


In [2]:

# Load the Iris dataset
iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',
                   header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'])
iris = iris.drop(columns='species')


In [3]:

# Define a probabilistic model using TensorFlow Probability
def model_fn():
    # Priors over the parameters of a Gaussian mixture model
    weights = tfp.layers.DenseFlipout(3, activation='softmax', name='weights')(tf.zeros((1, 4)))
    means = tfp.layers.DenseFlipout(3, name='means')(tf.zeros((1, 4)))
    scales = tfp.layers.DenseFlipout(3, activation='softplus', name='scales')(tf.zeros((1, 4)))

    # Mixture components
    dist = tfp.distributions.MixtureSameFamily(
        mixture_distribution=tfp.distributions.Categorical(probs=weights),
        components_distribution=tfp.distributions.Normal(loc=means, scale=scales))
    return dist


In [4]:

# Define a variational distribution using TensorFlow Probability
def variational_fn():
    # Approximate posterior over the parameters of the Gaussian mixture model
    weights = tfp.layers.VariableLayer(1, dtype=tf.float32, name='weights')
    means = tfp.layers.VariableLayer(1, dtype=tf.float32, name='means')
    scales = tfp.layers.VariableLayer(1, dtype=tf.float32, name='scales')
    return tfp.distributions.MixtureSameFamily(
        mixture_distribution=tfp.distributions.Categorical(probs=weights),
        components_distribution=tfp.distributions.Normal(loc=means, scale=scales))



In [5]:
# Create a TensorFlow Dataset object for the Iris data
data = tf.data.Dataset.from_tensor_slices((iris.values.astype(np.float32),))


In [6]:

# Create a TensorFlow Probability VI object for variational inference
vi = tfp.vi.fit_surrogate_posterior(
    target_log_prob_fn=lambda *args: model_fn().log_prob(args),
    surrogate_posterior=variational_fn(),
    optimizer=tf.optimizers.Adam(learning_rate=0.01),
    num_steps=1000,
    dataset=data)

ValueError: Attempt to convert a value (<tensorflow_probability.python.layers.variable_input.VariableLayer object at 0x7fb96273bd00>) with an unsupported type (<class 'tensorflow_probability.python.layers.variable_input.VariableLayer'>) to a Tensor.

In [ ]:

# Extract the posterior distribution over the parameters of the model
weights, means, scales = vi.surrogate_posterior.sample(1000).numpy().mean(axis=0)


In [ ]:


# Print the results
print('Inferred weights:', weights)
print('Inferred means:', means)
print('Inferred scales:', scales)
